# Load data

In [ ]:
# prompt: mount drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/regtr/regtr_liver.npz .

# Dataset and Dataloader

In [ ]:
import random
import torch
import numpy as np
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [ ]:
class LiverDataset(Dataset):
  def __init__(self, data:dict):
    super(LiverDataset, self).__init__()
    self.data = data
    self.keys = list(self.data.keys())
    self.device = 'cuda' if torch.cuda.is_available() else 'cpu'

  def __len__(self):
    return len(self.keys)

  def __getitem__(self, idx):
    return torch.from_numpy(self.data[self.keys[idx]]).to(self.device)

In [ ]:
data = dict(np.load('regtr_liver.npz'))
keys = list(data.keys())

In [ ]:
random.shuffle(keys)

In [ ]:
keys_tr = keys[:700]
keys_vl = keys[700:850]
keys_ts = keys[850:]

In [ ]:
dataset_tr = LiverDataset({k:data[k] for k in keys_tr})
dataset_vl = LiverDataset({k:data[k] for k in keys_vl})
dataset_ts = LiverDataset({k:data[k] for k in keys_ts})

In [ ]:
loader_tr = DataLoader(dataset_tr, batch_size=128, shuffle=True)
loader_vl = DataLoader(dataset_vl, batch_size=128, shuffle=True)
loader_ts = DataLoader(dataset_ts, batch_size=128, shuffle=True)

In [ ]:
for i in loader_tr:
  break

# Visualise

In [ ]:
!pip install open3d

In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go

In [ ]:
def arr2pcd(arr):
  pcd = o3d.geometry.PointCloud()
  pcd.points = o3d.utility.Vector3dVector(arr)
  return pcd

In [ ]:
pcd = arr2pcd(dataset_tr[100])

In [ ]:
pcd.colors

std::vector<Eigen::Vector3d> with 1000 elements.
Use numpy.asarray() to access data.

In [ ]:
# Adopted from https://colab.research.google.com/drive/1CR_HDvJ2AnjJV3Bf5vwP70K0hx3RcdMb?usp=sharing

def draw_geometries(geometries):
    graph_objects = []

    for geometry in geometries:
        geometry_type = geometry.get_geometry_type()

        if geometry_type == o3d.geometry.Geometry.Type.PointCloud:
            points = np.asarray(geometry.points)
            colors = None
            if geometry.has_colors():
                colors = np.asarray(geometry.colors)
            elif geometry.has_normals():
                colors = (0.5, 0.5, 0.5) + np.asarray(geometry.normals) * 0.5
            else:
                geometry.paint_uniform_color((0.0, 1.0, 0.0))
                colors = np.asarray(geometry.colors)

            scatter_3d = go.Scatter3d(x=points[:,0], y=points[:,1], z=points[:,2], mode='markers', marker=dict(size=1, color=colors))
            graph_objects.append(scatter_3d)

        # if geometry_type == o3d.geometry.Geometry.Type.TriangleMesh:
        #     triangles = np.asarray(geometry.triangles)
        #     vertices = np.asarray(geometry.vertices)
        #     colors = None
        #     if geometry.has_triangle_normals():
        #         colors = (0.5, 0.5, 0.5) + np.asarray(geometry.triangle_normals) * 0.5
        #         colors = tuple(map(tuple, colors))
        #     else:
        #         colors = (1.0, 0.0, 0.0)

        #     mesh_3d = go.Mesh3d(x=vertices[:,0], y=vertices[:,1], z=vertices[:,2], i=triangles[:,0], j=triangles[:,1], k=triangles[:,2], facecolor=colors, opacity=0.50)
        #     graph_objects.append(mesh_3d)
d
    fig = go.Figure(
        data=graph_objects,
        layout=dict(
            scene=dict(
                xaxis=dict(visible=False),
                yaxis=dict(visible=False),
                zaxis=dict(visible=False)
            )
        )
    )
    fig.show()

o3d.visualization.draw_geometries = draw_geometries # replace function


In [ ]:
o3d.visualization.draw_geometries([arr2pcd(dataset_tr[0])])

# Detect planes

Here it uses the point cloud.

Alternatives: perhaps easier on the voxel data? Sum along one axis and see any sudden cut-off.

## One plane

The point cloud above has a cut plane z=-1. Let's see whether the `.segment_plane()` can find the plane.

Code adapted from https://www.open3d.org/docs/latest/tutorial/Basic/pointcloud.html#Plane-segmentation

Remains to be validated for multiple planes.

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")



Plane equation: 0.43x + 0.07y + 0.90z + -0.11 = 0


In [ ]:
len(inliers)

134

In [ ]:
# Visualise the plane
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])

## More than one plane

In [ ]:
# Three planes: x=1, y=1, z=1
key = 's0009_liver'

In [ ]:
key in keys_vl

True

In [ ]:
keys_vl.index(key)

105

In [ ]:
pcd = arr2pcd(dataset_vl[105])

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: -0.00x + -0.00y + 1.00z + -1.00 = 0


It finds the largest plane.

In [ ]:
len(inliers)

295

## Complete liver but with clear layers of voxels

In [ ]:
key = 's0011_liver'

In [ ]:
key in keys_ts

True

In [ ]:
keys_ts.index(key)

6

In [ ]:
pcd = arr2pcd(dataset_ts[6])

In [ ]:
o3d.visualization.draw_geometries([pcd])

In [ ]:
plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
[a, b, c, d] = plane_model
print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

Plane equation: 0.58x + 0.21y + 0.79z + -0.11 = 0


In [ ]:
len(inliers)

135

In [ ]:
# Visualise the plane
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])
outlier_cloud = pcd.select_by_index(inliers, invert=True)
o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])